In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel
from keras.models import load_model

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.10.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'lm_head.weight', 'transformer.h.5.attn.masked_bias', 'transformer.h.6.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expe

In [5]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print('-' * 10)
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

1
1
3
----------
</s>
<usr>
<pad>
<sys>


In [6]:
import pandas as pd
import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

In [7]:
BASE_PATH = '/content/drive/MyDrive/CP2_NLP/'

In [8]:
def get_chat_data():
  for person, system in zip(train_data.P.to_list(), train_data.S.to_list()):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode('<usr>' + person + '<sys>' + system) 
    yield bos_token + sent + eos_token

for i in tqdm.tqdm_notebook(os.listdir(BASE_PATH+'/Data/')):
    if i =='감성대화말뭉치.xlsx':
        pass
    elif i == 'sad_df.csv':
        pass
    elif i == 'neutrality_df.csv':
        pass
    else:
        train_data = pd.read_csv(BASE_PATH+f'/Data/{i}')
        
        batch_size = 16
        dataset = tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32)
        dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

        adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
        steps = len(train_data) // batch_size + 1

        
        EPOCHS = 3
        for epoch in range(EPOCHS):
            epoch_loss = 0

            for batch in tqdm.tqdm_notebook(dataset, total=steps):
                with tf.GradientTape() as tape:
                    result = model(batch, labels=batch)
                    loss = result[0]
                    batch_loss = tf.reduce_mean(loss)
                
                grads = tape.gradient(batch_loss, model.trainable_variables)
                adam.apply_gradients(zip(grads, model.trainable_variables))
                epoch_loss += batch_loss / steps

            print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

        model.save_pretrained(BASE_PATH+f'Model/Model Save/KoGPT-2/KoGPT-2_{i[:-7]}_model.h5')
                

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/1134 [00:00<?, ?it/s]

[Epoch:    1] cost = 1.82899261


  0%|          | 0/1134 [00:00<?, ?it/s]

[Epoch:    2] cost = 1.53290939


...

[Epoch:    1] cost = 1.80496693


  0%|          | 0/1081 [00:00<?, ?it/s]

[Epoch:    2] cost = 1.46547449


  0%|          | 0/1081 [00:00<?, ?it/s]

[Epoch:    3] cost = 1.07753336


# Test

In [9]:
def return_answer_by_chatbot(user_text, emo_model):
  sent = '<usr>' + user_text + '<sys>'
  input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
  input_ids = tf.convert_to_tensor([input_ids])
  output = emo_model.generate(input_ids, max_length=50, do_sample=True, top_k=20)
  sentence = tokenizer.decode(output[0].numpy().tolist())
  chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '')
  return chatbot_response

In [10]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
ha_model = TFGPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/CP2_NLP/Model/Model Save/KoGPT-2/KoGPT-2_happy_model.h5')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/drive/MyDrive/CP2_NLP/Model/Model Save/KoGPT-2/KoGPT-2_happy_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [16]:
sa_model = TFGPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/CP2_NLP/Model/Model Save/KoGPT-2/KoGPT-2_sad_model.h5')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/drive/MyDrive/CP2_NLP/Model/Model Save/KoGPT-2/KoGPT-2_sad_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [13]:
return_answer_by_chatbot('이번에 건강검진 결과가 나왔는데, 아주 좋아.', ha_model)

'앞으로도 건강하시길 바라요.'

In [14]:
return_answer_by_chatbot('나 남자친구 생겼어.', ha_model)

'축하드려요.'

In [15]:
return_answer_by_chatbot('나 오늘 너무 우울해.', ha_model)

'우울하시다니 많이 힘내셨나보네요.'

In [17]:
return_answer_by_chatbot('나 오늘 너무 우울해.', sa_model)

'하루하루가 힘디도 고마운 마음을 모두 기억해 주시길 바라요.'

In [18]:
return_answer_by_chatbot('나 오늘 너무 속상해.', sa_model)

'좋은 결과 있길 바랄게요.'